In [ ]:
from ultralytics import YOLO
import numpy as np
import cv2
import pickle
import catcher as CT
import SaveSystem as SS
import torch
print(torch.cuda.is_available())

surface_model = YOLO("cube_surface_seg2.pt")
cube_model = YOLO("yolov8n-seg-custom.pt")

with open('./hand_matrix/calibration.pkl', 'rb') as file:
    camera_matrix, dist_coeff = pickle.load(file)
# print(camera_matrix, dist_coeff)
# with open('./hand_matrix/camMatrix.npy', 'rb') as file:
#     m_camera_matrix = np.load(file)
# with open('./hand_matrix/distCoef.npy', 'rb') as file:
#     m_dist_coeff = np.load(file)
# print(m_camera_matrix, m_dist_coeff)
    
CT = CT.block_detect(surface_model=surface_model, cube_model=cube_model)

In [ ]:
#HSV
import cv2
def empty(v):
    pass

block = cv2.imread('./images/img5.png')
block = cv2.resize(block, (0, 0), fx=.7, fy=.7)

cv2.namedWindow('TrackBar')
cv2.resizeWindow('TrackBar', 640, 320)

cv2.createTrackbar('Hue Min', 'TrackBar', 0, 179, empty)
cv2.createTrackbar('Hue Max', 'TrackBar', 179, 179, empty)
cv2.createTrackbar('Sat Min', 'TrackBar', 0, 255, empty)
cv2.createTrackbar('Sat Max', 'TrackBar', 255, 255, empty)
cv2.createTrackbar('Val Min', 'TrackBar', 0, 255, empty)
cv2.createTrackbar('Val Max', 'TrackBar', 255, 255, empty)

hsv = cv2.cvtColor(block,cv2.COLOR_BGR2HSV)
while True:
    h_min = cv2.getTrackbarPos('Hue Min', 'TrackBar')
    h_max = cv2.getTrackbarPos('Hue Max', 'TrackBar')
    s_min = cv2.getTrackbarPos('Sat Min', 'TrackBar')
    s_max = cv2.getTrackbarPos('Sat Max', 'TrackBar')
    v_min = cv2.getTrackbarPos('Val Min', 'TrackBar')
    v_max = cv2.getTrackbarPos('Val Max', 'TrackBar')
    # print(h_min, h_max, s_min, s_max, v_min, v_max)
    
    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(block, block, mask=mask)

    cv2.imshow('block', block)
    # cv2.imshow('hsv', hsv)
    cv2.imshow('mask', mask)
    cv2.imshow('result', result)
    

    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
hsv_boundary = {
    "red": np.array([[0, 150, 0], [20, 255, 255]], dtype=np.uint8),
    "blue": np.array([[0, 0, 0], [179, 70, 80]], dtype=np.uint8),
    "green": np.array([[38, 33, 0], [75, 165, 255]], dtype=np.uint8),
    "yellow": np.array([[20, 50, 110], [50, 255, 255]], dtype=np.uint8),
    "purple": np.array([[100, 0, 55], [160, 255, 160]], dtype=np.uint8)
}

In [ ]:
#抓圖片
img = cv2.imread("block_2.jpg")
# img = cv2.imread("123.jpg")
img = cv2.resize(img, (640, 480))

object_points = np.array([
    [0,0,0],#1
    [0,25,0],#2
    [25,25,0],#3
    [25,0,0],#4
    [0,0,25],#5
    [0,25,25],#6
    [25,25,25],#7
    [25,0,25],#8
], dtype=np.float32)

text_offset = 0

text_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
for image_points in CT.detect_parts(img, 2):
    # image_points = np.array(image_points, dtype=np.float32)
    print(len(image_points))
    for color_name, rgb in CT.get_color_text(img):
        image_points = np.float32(image_points)
        retval, rvec, tvec = cv2.solvePnP(object_points[:4], image_points, camera_matrix, dist_coeff)
        z = tvec[2]
        x = tvec[0]
        y = tvec[1] 

        rotation_matrix, _ = cv2.Rodrigues(rvec)

        # 使用旋转矩阵计算欧拉角（roll-pitch-yaw 顺序）
        yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
        pitch = np.arctan2(-rotation_matrix[2, 0], np.sqrt(rotation_matrix[2, 1]**2 + rotation_matrix[2, 2]**2))
        roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])

        # 将弧度转换为度数
        rx = np.degrees(yaw)
        ry = np.degrees(pitch)
        rz = np.degrees(roll)

        xyz_str = [f"{c}: {v[0]:.2f}" for c, v in zip("xyz", [x, y, z])]
        text_rotation = (5, 15 + text_offset)
        text_position = (5, 15 + text_offset)
        cv2.putText(text_img, f"{color_name} {', '.join(xyz_str)}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
        # cv2.putText(text_img, f"Yaw: {yaw_deg:.1f}, Pitch: {pitch_deg:.1f}, Roll: {roll_deg:.1f}", text_rotation, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
        result_image = cv2.addWeighted(img, 1, text_img, 1, 0)
        text_offset += 17

cv2.imshow('img', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def transform_matrix(x, y, z, rx, ry, rz):
    # 轉換為弧度
    rx, ry, rz = np.radians(rx), np.radians(ry), np.radians(rz)

    # 計算旋轉矩陣
    rotation_matrix = R.from_euler('xyz', [rx, ry, rz], degrees=False).as_matrix()

    # 構建轉移矩陣
    transformation_matrix = np.vstack([np.hstack([rotation_matrix, np.array([[x], [y], [z]])]),
                                      np.array([0, 0, 0, 1])])

    return transformation_matrix


In [5]:
import numpy as np

def rotation_x(theta):
    return np.array([[1, 0, 0],
                     [0, np.cos(theta), -np.sin(theta)],
                     [0, np.sin(theta), np.cos(theta)]])

def rotation_y(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                     [0, 1, 0],
                     [-np.sin(theta), 0, np.cos(theta)]])

def rotation_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                     [np.sin(theta), np.cos(theta), 0],
                     [0, 0, 1]])

def transform_matrix(x, y, z, rx, ry, rz):
    rx = np.radians(rx)
    ry = np.radians(ry)
    rz = np.radians(rz)

    rotation_matrix = np.dot(rotation_z(rz), np.dot(rotation_y(ry), rotation_x(rx)))
    translation_vector = np.array([[x], [y], [z]])

# 合併旋轉矩陣和平移向量
    transformation_matrix = np.vstack([np.hstack([rotation_matrix, translation_vector]),
                                      np.array([0, 0, 0, 1])])

    return transformation_matrix

In [51]:
lens = cv2.VideoCapture(0,cv2.CAP_DSHOW)

In [50]:
lens.release()

In [53]:
# object_points = np.array([
#     [-12.5,-12.5,0],#1
#     [-12.5,12.5,0],#2
#     [12.5,12.5,0],#3
    # [12.5,-12.5,0],#4
# ], dtype=np.float32)
object_points = np.array( #中心
    [
        [-25, -25, 0],  # 1
        [-25, 25, 0],  # 2
        [25, 25, 0],  # 3
        [25, -25, 0],  # 4
    ],
    dtype=np.float32,
)
# object_points = np.array([
#     [-12.5,-12.5,0],#1
#     [-12.5,12.5,0],#2
#     [12.5,12.5,0],#3
#     [12.5,-12.5,0],#4
# ], dtype=np.float32)
object_points = np.array(
    [
        [-25, -25, 0],  # 1
        [-25, 25, 0],  # 2
        [25, 25, 0],  # 3
        [25, -25, 0],  # 4
    ],
    dtype=np.float32,
)
SS.SaveSystem.reset()
offset_x = 0
offset_y = 0

while True:
    ret, frame = lens.read()
    if not ret:
        break
    vertical_offset = 0
    for image_points in CT.detect_parts(frame):
        for color_name, rgb in CT.get_color_text(frame):
            image_points = np.float32(image_points)
            retval, rvec, tvec = cv2.solvePnP(object_points[:4], image_points, camera_matrix, dist_coeff)
            if not retval:
                break
            x = tvec[0] - offset_x
            y = tvec[1] - offset_y
            z = tvec[2]
            rotation_matrix, _ = cv2.Rodrigues(rvec)
            # 使用旋转矩阵计算欧拉角（roll-pitch-yaw 顺序）
            yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
            pitch = np.arctan2(
                -rotation_matrix[2, 0],
                np.sqrt(rotation_matrix[2, 1] ** 2 + rotation_matrix[2, 2] ** 2),
            )
            roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
            # 将弧度转换为度数
            rx0 = np.degrees(yaw)
            ry = np.degrees(pitch)
            rz = np.degrees(roll)
            RotationZ = np.array([rx0])
            text_loc_tvec = (5, 15 + vertical_offset)
            text_loc_rvec = (5, 32 + vertical_offset)
            text_loc_check = (400, 15 + vertical_offset)
            # print(color_name)
            print(f"{x=}")
            print(f"{y=}")
            print(f"{rz=}")
            SS.SaveSystem.save_coordinate(color_name, x, y,RotationZ)
            xyz_str = [f"{c}: {v[0]:.2f}" for c, v in zip("xyz", [x, y, z])]
            cv2.putText(frame, f"{color_name} {', '.join(xyz_str)}", text_loc_tvec, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
            cv2.putText(frame, f"Rotate Z: {rx0:.1f},   Rotate Y: {ry:.1f},   Rotate X: {rz:.1f}", text_loc_rvec, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
            if -10 < x < 10 and -10 < y < 10:
                cv2.putText(frame, "OK", text_loc_check, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Moving", text_loc_check, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255) ,2)
            cv2.circle(frame, (int(x[0]), int(y[0])), 5, (0, 0, 255), -1)

    else:
        pass
    cv2.imshow("test", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cv2.destroyAllWindows()

x=array([    -7.3189])
y=array([     -8.722])
rz=-4.957324693949771
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
yellow，已達上限
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
x=array([    -6.9566])
y=array([    -2.2454])
rz=-11.628732939248023
x=array([    -6.9566])
y=array([    -2.2454])
rz=-11.628732939248023
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
x=array([    -6.8686])
y=array([    -2.2458])
rz=-12.275639234860362
x=array([    -6.8686])


In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R


def calculate_transform_matrix(initial_position, final_position):
    # 初始座標和移動後的座標
    p1 = np.array(initial_position)
    p2 = np.array(final_position)

    # 計算平移向量
    translation_vector = p2 - p1

    # 計算旋轉矩陣
    direction = p2 - p1
    axis = np.cross(np.array([1, 0, 0]), direction)
    angle = np.arccos(np.dot(np.array([1, 0, 0]), direction) / (np.linalg.norm(direction)))

    rotation_matrix = R.from_rotvec(axis * angle).as_matrix()

    # 構建轉移矩陣
    transformation_matrix = np.vstack([np.hstack([rotation_matrix, translation_vector.reshape(3, 1)]),
                                      np.array([0, 0, 0, 1])])

    return transformation_matrix
np.linalg.inv(transform_matrix(200,500,509,-180, 0, -60)) @ calculate_transform_matrix([0,0,509],[200,500,509])

In [3]:
coor = SS.SaveSystem.get_coordinates_by_color("yellow")
print(coor)

[[     228.58      505.85      -36.67]
 [     228.64      502.75      -49.79]
 [     229.99      505.32      -38.56]
 [     229.99      505.32      -38.56]]
